In [7]:
import numpy as np 
import pandas as pd 

In [8]:
def dia_del_any(data):
    resultat = data.dt.day + data.dt.month*12
    return resultat

In [10]:
def netejar_dades():

    df = pd.read_excel("consumo_material_clean.xlsx")
    df.head()
    df['NUMERO'] = df['NUMERO'].str.split('/')
    df['NUMERO'] = df['NUMERO'].str[0]
    df['CATEGORIA'] = df['CODIGO'].str[0]
    df['CODIGO'] = df['CODIGO'].str[1::]
    df['ANY'] = pd.to_datetime(df['FECHAPEDIDO'], format='mixed').dt.year
    df['ORIGEN'] = df['ORIGEN'].str.split('-')
    df['REGION_ORIGEN'] = df['ORIGEN'].str[0]
    df['HOSPITAL_ORIGEN'] = df['ORIGEN'].str[1]
    df['DEPARTAMENT_ORIGEN'] = df['ORIGEN'].str[2]
    return df

df=netejar_dades() 
df.head()